In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from utils_cd import (
        split_dataset,
        standard_deviation,
        plot_comparison_results,
        impute_values,
        plot_results,
        plot_param_improv,
        plot_results_from_csv,
        aps_classifier_statistics,
        aps_score,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import Normalizer, StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.base import clone
from sklearn import tree
import graphviz

pd.options.display.max_columns = None
sns.set(style='darkgrid')

CLASS = 'class'

def getData():
    train = pd.read_csv('./train.csv')
    test = pd.read_csv('./test.csv')

    X_train, y_train = split_dataset(train, CLASS)
    X_test, y_test = split_dataset(test, CLASS)
    y_train = y_train.map({'pos': 1, 'neg': 0})
    y_test = y_test.map({'pos': 1, 'neg': 0})
    
    return X_train, X_test, y_train, y_test

def getDataWithThresh():
    train = pd.read_csv('./train.csv')
    test = pd.read_csv('./test.csv')

    train = train.dropna(thresh=150)
    train = train.dropna(axis=1, thresh=train.shape[0]*0.9)
    test = test[train.columns]

    X_train, y_train = split_dataset(train, CLASS)
    X_test, y_test = split_dataset(test, CLASS)
    y_train = y_train.map({'pos': 1, 'neg': 0})
    y_test = y_test.map({'pos': 1, 'neg': 0})
    
    return X_train, X_test, y_train, y_test

data = {}
clf = RandomForestClassifier(random_state=42)
min_samples = np.concatenate([np.arange(5, 51, 5), np.arange(100, 1001, 100)])
num_features = np.arange(10, 170, 3)
results = {}
res_i = 0

In [2]:
X_train, X_test, y_train, y_test = getData()
X_train, X_test = X_train.fillna(X_train.median()), X_test.fillna(X_train.median())
def balance(X_train, y_train):
    balancer = RandomUnderSampler(ratio=0.3, random_state=42)
    X_train_bal, y_train_bal = balancer.fit_sample(X_train, y_train)
    return X_train_bal, y_train_bal

X_train, y_train = balance(X_train, y_train)


/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
for samples in min_samples:
    res = aps_classifier_statistics(RandomForestClassifier(min_samples_split=samples), X_train, X_test, y_train, y_test)
    res_train = aps_classifier_statistics(RandomForestClassifier(min_samples_split=samples), X_train, X_train, y_train, y_train)
    results[res_i] = {'Test': res['auc'], 'Min Samples to Split': samples}
    res_i += 1
    results[res_i] = {'Train': res_train['auc'], 'Min Samples to Split': samples}
    res_i += 1
    print(res)
    print(res_train)

/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


{'predicted': array([0, 0, 0, ..., 0, 0, 0]), 'accuracy': 0.969561403508772, 'confusion_matrix': array([[10881,   322],
       [   25,   172]]), 'sensibility': 0.8730964467005076, 'specificity': 0.9712576988306704, 'auc': 0.9221770727655889, 'clf': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), 'score': 15720}
{'predicted': array([0, 0, 0, ..., 1, 1, 1]), 'accuracy': 0.9935513588208199, 'confusion_matrix': array([[1663,    7],
       [   7,  494]]), 'sensibility': 0.9860279441117764, 'specificity': 0.9958083832335329, 'auc': 0.9909181636726546, 'clf': RandomForestClassifier(bootstrap=True, class_weight=None, criter

/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


{'predicted': array([0, 0, 0, ..., 0, 0, 0]), 'accuracy': 0.968421052631579, 'confusion_matrix': array([[10865,   338],
       [   22,   175]]), 'sensibility': 0.8883248730964467, 'specificity': 0.9698295099526912, 'auc': 0.9290771915245689, 'clf': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), 'score': 14380}
{'predicted': array([0, 0, 0, ..., 1, 1, 1]), 'accuracy': 0.9847996315062183, 'confusion_matrix': array([[1653,   17],
       [  16,  485]]), 'sensibility': 0.9680638722554891, 'specificity': 0.9898203592814371, 'auc': 0.978942115768463, 'clf': RandomForestClassifier(bootstrap=True, class_weight=None, criter

/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

{'predicted': array([0, 0, 0, ..., 0, 0, 0]), 'accuracy': 0.9521052631578948, 'confusion_matrix': array([[10674,   529],
       [   17,   180]]), 'sensibility': 0.9137055837563451, 'specificity': 0.9527805052218156, 'auc': 0.9332430444890804, 'clf': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=700,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), 'score': 13790}
{'predicted': array([0, 1, 0, ..., 1, 1, 1]), 'accuracy': 0.954398894518655, 'confusion_matrix': array([[1589,   81],
       [  18,  483]]), 'sensibility': 0.9640718562874252, 'specificity': 0.951497005988024, 'auc': 0.9577844311377246, 'clf': RandomForestClassifier(bootstrap=True, class_weight=None, crite

/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


{'predicted': array([0, 0, 0, ..., 0, 0, 0]), 'accuracy': 0.9514035087719298, 'confusion_matrix': array([[10663,   540],
       [   14,   183]]), 'sensibility': 0.9289340101522843, 'specificity': 0.9517986253682049, 'auc': 0.9403663177602445, 'clf': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=800,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), 'score': 12400}
{'predicted': array([0, 1, 0, ..., 1, 1, 1]), 'accuracy': 0.9567019806540764, 'confusion_matrix': array([[1591,   79],
       [  15,  486]]), 'sensibility': 0.9700598802395209, 'specificity': 0.9526946107784431, 'auc': 0.961377245508982, 'clf': RandomForestClassifier(bootstrap=True, class_weight=None, crit

/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/francisconeves/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


{'predicted': array([0, 0, 0, ..., 0, 0, 0]), 'accuracy': 0.9535964912280702, 'confusion_matrix': array([[10687,   516],
       [   13,   184]]), 'sensibility': 0.934010152284264, 'specificity': 0.9539409086851736, 'auc': 0.9439755304847188, 'clf': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=900,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), 'score': 11660}
{'predicted': array([0, 1, 0, ..., 1, 1, 1]), 'accuracy': 0.9557807461999078, 'confusion_matrix': array([[1589,   81],
       [  15,  486]]), 'sensibility': 0.9700598802395209, 'specificity': 0.951497005988024, 'auc': 0.9607784431137725, 'clf': RandomForestClassifier(bootstrap=True, class_weight=None, crite

In [4]:
measures = pd.DataFrame.from_dict(results, "index")
measures = measures.set_index('Min Samples to Split')
filename = 'rf_min_samples'
df = pd.DataFrame()
for col in measures:
    if col != 'Min Samples to Split':
        values = measures[col].dropna()
        if df.empty:
            df = measures[[col]].copy()
        else:
            df = pd.concat([df, values], axis=1, join='inner')
        
df.to_csv('plot_data/{}.csv'.format(filename))

In [ ]:
X_train, X_test, y_train, y_test = getData()
X_train, X_test = X_train.fillna(X_train.median()), X_test.fillna(X_train.median())
def balance(X_train, y_train):
    balancer = RandomUnderSampler(ratio=0.3, random_state=42)
    X_train_bal, y_train_bal = balancer.fit_sample(X_train, y_train)
    return X_train_bal, y_train_bal

X_train, y_train = balance(X_train, y_train)

def getBaseline():
    X_train, X_test, y_train, y_test = getData()
    X_train, X_test = X_train.fillna(X_train.median()), X_test.fillna(X_train.median())
    return X_train, X_test, y_train, y_test

def getDataFinal():
    X_train, X_test, y_train, y_test = getData()
    X_train, X_test = X_train.fillna(X_train.median()), X_test.fillna(X_train.median())
    def balance(X_train, y_train):
        balancer = RandomUnderSampler(ratio=0.3, random_state=42)
        X_train_bal, y_train_bal = balancer.fit_sample(X_train, y_train)
        return X_train_bal, y_train_bal

    X_train, y_train = balance(X_train, y_train)
    return X_train, X_test, y_train, y_test